# Explore Model Instance
After a model is run, this notebook can be used to explore the model instance and results, if exported as pickle files.

To export the model instance as a pickle file, `--save-instance` must be added to `options.txt` (NOTE: this will be a large file!)  
Switch will automatically export the model results as a pickle file unless the `--no-save-solution` is specified in `options.txt`  

In order to explore duals and reduced costs, `-suffixes dual rc` must be added to `options.txt`

In [ ]:
from pathlib import Path
import pickle
import cloudpickle
import pyomo.environ as pyo
import pandas as pd

# specify where the pickle file is located
model_path = '../MODEL_RUNS/test_PCE/outputs/hourly_90/'

# Explore Model Instance

In [ ]:
# read instance file
with open((Path.cwd() / model_path / 'instance.pickle'), mode='rb') as file:
    instance = cloudpickle.load(file)

In [ ]:
len(instance.GENERATION_PROJECTS)

## Explore Sets

In [ ]:
i = 0
for setobject in instance.component_objects(pyo.Set, active=True):
    nametoprint = str(str(setobject.name))
    if '_index' in nametoprint:
        pass
    else:
        print ("Set ", nametoprint)  
        i+=1
print(f'Total Number of sets: {i}')

In [ ]:
# to examine the set
instance.GENS_BY_TECHNOLOGY.pprint()

## Explore Parameters, Expressions, and Vars

In [ ]:
for ExpObject in instance.component_objects(pyo.Expression, active=True):
    nametoprint = str(str(ExpObject.name))
    print ("Expression ", nametoprint) 

In [ ]:
instance.DLAPLoadCostInTP.pprint()

In [ ]:
pyo.value(instance.BuildGen['PVHYBRID_Chaparral_Springs', 2025])

In [ ]:
# print all parameters
i=0
for parmobject in instance.component_objects(pyo.Param, active=True):
    nametoprint = str(str(parmobject.name))
    print ("Parameter ", nametoprint)  
    i+=1
print(f'Total Number of parameters: {i}')

In [ ]:
instance.add_one_to_period_end.pprint()

## Explore Constraints
I might not be able to access constraint values: https://stackoverflow.com/questions/50703321/how-to-retrieve-value-of-constraint-from-pyomo

In [ ]:
# list all of the constraints
for c in instance.component_objects(pyo.Constraint, active=True):
    print ("Constraint",c)

In [ ]:
instance.Zone_Energy_Balance.pprint()

In [ ]:
# can get numerical values of single constraint through lower, body, upper
c = instance.Zone_Energy_Balance
print ("   Constraint",c)
i = 1
for index in c:
    if i < 5:
        print ("      ", index, c[index].upper)
        i+=1
    else:
        break

## Explore Slack Values

In [ ]:
# https://pyomo.readthedocs.io/en/stable/working_models.html#accessing-slacks

c = instance.Zone_Energy_Balance
print ("   Constraint",c)
for index in c:
    print ("      ", index, round(c[index].lslack(),4), round(c[index].uslack(),4))


In [ ]:
instance.dual[instance.Zone_Energy_Balance[('DLAPPGAE', 6789)]]

# Explore Results

The SolverResults object contains four main pieces of data:
 - Problem
 - Solver
 - Solution
 - Pyomo solve time

In [ ]:
# read results file
with open((Path.cwd() / model_path / 'results.pickle'), 'rb') as file:
    results = pickle.load(file)

print(results.problem)

In [ ]:
print(results.solver)

In [ ]:
print(results.pyomo_solve_time)

In [ ]:
print(results.solution)

## Explore Reduced Costs

In [ ]:
variable = 'BuildGen'
variable_rc = pd.DataFrame.from_dict(results.solution.Variable, orient='index')
variable_rc = variable_rc.reset_index()
variable_rc[['Variable','index']] = variable_rc['index'].str.split('[', expand=True)
variable_rc['index'] = variable_rc['index'].str.strip(']')

variable_rc = variable_rc[['Variable','index','Value','Rc']]

variable_rc = variable_rc[variable_rc['Variable'] == variable]

# split the index into the load zone and timepoint components
variable_rc[['generation_project','period']] = variable_rc['index'].str.split(',', expand=True)
variable_rc = variable_rc.drop(columns=['period','index'])

"""
# load the variable capacity factor data
vcf = pd.read_csv('../MODEL_RUNS/test_PCE/inputs/hourly_90/variable_capacity_factors.csv')

# sum capacity factors by generator
vcf = vcf.groupby('GENERATION_PROJECT').sum()[['variable_capacity_factor']].reset_index()

gen_list = list(variable_rc['generation_project'].unique())

for g in gen_list:
    variable_rc.loc[variable_rc['generation_project'] == g, 'Rc'] = variable_rc.loc[variable_rc['generation_project'] == g, 'Rc'].item() / vcf.loc[vcf['GENERATION_PROJECT'] == g, 'variable_capacity_factor'].item()
"""
variable_rc

In [ ]:
g = 'WAVE_Calwave'
variable_rc.loc[variable_rc['generation_project'] == g, 'Rc'].item() / vcf.loc[vcf['GENERATION_PROJECT'] == g, 'variable_capacity_factor'].item()

In [ ]:
variable_rc.loc[variable_rc['generation_project'] == g, 'Rc']

## Explore Duals

In [ ]:
constraint = 'Zone_Energy_Balance'

# load the duals into a dataframe and reset the index
constraint_duals = pd.DataFrame.from_dict(results.solution.Constraint, orient='index')
constraint_duals = constraint_duals.reset_index()

# split the index into columns for the constraint name and the index value
constraint_duals[['Constraint','index']] = constraint_duals['index'].str.split('[', expand=True)
#constraint_duals['index'] = '[' + constraint_duals['index']
constraint_duals['index'] = constraint_duals['index'].str.strip(']')

# filter the constraints to the zone energy balance
constraint_duals = constraint_duals[constraint_duals['Constraint'] == constraint]

# split the index into the load zone and timepoint components
constraint_duals[['load_zone','timepoint']] = constraint_duals['index'].str.split(',', expand=True)
constraint_duals['timepoint'] = constraint_duals['timepoint'].astype(int)
constraint_duals = constraint_duals.drop(columns=['index'])

# sort the values
constraint_duals = constraint_duals.sort_values(by=['load_zone','timepoint'])

#re-order columns
constraint_duals = constraint_duals[['Constraint', 'load_zone','timepoint','Dual']]


constraint_duals

In [ ]:
import plotly.express as px

px.line(constraint_duals, x='timepoint', y='Dual')